In [24]:
from dotenv import load_dotenv
import os
import pandas as pd
from sqlalchemy import create_engine, text
from datetime import datetime as dt
import logging
logging.basicConfig(level=logging.INFO)
logger = logging.getLogger(__name__)
load_dotenv()



def get_footprint_data_from_db(reg_number):
    try:
        db_host = os.getenv("DB_HOST_UAT")
        db_port = os.getenv("DB_PORT")
        db_name = os.getenv("FP_DB_NAME")
        driver = os.getenv("DRIVER")
        trusted_conn = os.getenv("TRUSTED_CONN")
        server = os.getenv("SERVER")

        connection_string = (
            f"mssql+pyodbc://{db_host},{db_port}/{db_name}?driver={driver}&trusted_connection={trusted_conn}"
        )
        #SQLAlchemy engine
        engine = create_engine(connection_string)
        logger.info(f"Successfully connected to the footprint DB::")
    except Exception as e:
        logger.error('error getting DB credentials:: {e}')

    try:
        #query
        with engine.connect() as connection:
            query = text(f'''SELECT
                                top 5 * 
                            FROM ICEALIONVehiclesnew
                            WHERE RegistrationNo  LIKE ('%KAD%070%');
                        ''')
            df = pd.read_sql_query(
                query,
                connection
            )
        return df
    except Exception as e:
        logger.error(f'error fetching FP_data{e}')

In [25]:
df = get_footprint_data_from_db('KAD 070')

INFO:__main__:Successfully connected to the footprint DB::


In [27]:
df.columns

Index(['RegistrationNo', 'PolicyNumber', 'Insured', 'PeriodFrom', 'PeriodTo',
       'MotorVehicleCode', 'CertificateNumber', 'RevenueType', 'ChassisNo',
       'EngineNo', 'LogBookNo', 'VehicleMake', 'VehicleMakeCode',
       'VehicleModel', 'VehicleModelCode', 'BodyType', 'BodyTypeCode',
       'YearOfManufacture', 'CubicCapacity', 'Color', 'ColorCode',
       'EstimatedValue', 'RadioValue', 'WindscreenValue', 'NCDYears',
       'VehiclePurpose', 'VehiclePurposeCode', 'CarryingCapacity',
       'VehicleConditionStatus', 'VehicleStatus', 'Company', 'ItemUk',
       'BasicPremium', 'RenewalPremium', 'RateApplied', 'Brokercode',
       'BrokerName', 'RenewalDate', 'EffectiveDate', 'ExpiryDate',
       'NumberOfMonths', 'ClassCode', 'ClassDesc', 'SchemeName', 'NCDDiscount',
       'DepartCode', 'DepartmentName', 'ValuationSent', 'ValuationDone',
       'LetterRef', 'DateofBirth', 'SittingCapacity', 'ItemKey', 'Brokerno',
       'schemecode', 'Schemedesc', 'AgencyType', 'Branchcode', 'Bra

In [46]:
import random
from datetime import datetime, timedelta

random.seed(42)  # For reproducibility

# Helper to generate random dates
def random_date(start, end):
    delta = end - start
    days = random.randint(0, delta.days)
    return (start + timedelta(days=days)).strftime('%Y-%m-DD')

start_date = datetime(2024, 1, 1)
end_date = datetime(2025, 10, 21)

# existing_db (db_A)
existing_db = {
    'table_1': {
        'id': list(range(1, 28)) + [1, 2, 3],  # 27 unique + 3 duplicates
        'names': random.choices(['John Smith', 'Alice Johnson', 'Bob Lee', 'Emma Davis', 'David Kim'], k=30),
        'Amount': [round(random.uniform(50.0, 500.0), 2) for _ in range(30)],
        'date': [random_date(start_date, end_date) for _ in range(30)]
    },
    'table_2': {
        'order_id': list(range(101, 128)) + [101, 102, 103],  # 27 unique + 3 duplicates
        'product': random.choices(['Laptop', 'Phone', 'Tablet', 'Headphones'], k=30),
        'price': [round(random.uniform(20.0, 1000.0), 2) for _ in range(30)],
        'quantity': random.choices([1, 2, 3, 4, 5], k=30)
    }
}

# new_db (with mismatches)
new_db = {
    'table_1': {
        'new_id': list(range(1, 27)) + [1, 2, 3, 4],  # 26 unique + 4 duplicates (mismatch)
        'new_names': random.choices(['John Smith', 'Alice Johnson', 'Bob Lee', 'Emma Davis', 'David Kim', 'Sara Wong'], k=30),  # Added new name
        'new_Amount': [round(random.uniform(50.0, 500.0), 2) if random.random() > 0.1 else round(random.uniform(40.0, 600.0), 2) for _ in range(30)],  # Some value changes
        'new_date': [random_date(start_date, end_date) for _ in range(30)]  # Different dates
    },
    'table_2': {
        'new_order_id': list(range(101, 127)) + [101, 102, 103, 104],  # 26 unique + 4 duplicates
        'new_product': random.choices(['Laptop', 'Phone', 'Tablet', 'Headphones', 'Speaker'], k=30),  # Added new product
        'new_price': [round(random.uniform(20.0, 1000.0), 2) if random.random() > 0.1 else round(random.uniform(15.0, 1200.0), 2) for _ in range(30)],  # Some value changes
        'new_quantity': random.choices([1, 2, 3, 4, 5, 6], k=30)  # Extended range
    }
}
pd.DataFrame(new_db['table_1']).sample()

,new_id,new_names,new_Amount,new_date
18,19,Sara Wong,304.35,2024-12-DD


In [ ]:
import numpy as np 
new_df1 = pd.DataFrame(new_db['table_1'])
ex_df1 = pd.DataFrame(existing_db['table_1'])

col_mappings = {'id':'new_id', 'names':'new_names', 
                'Amount':'new_Amount', 'date':'new_date'}
tables = [ex_df1, new_df1]

cleaning_features_table = []   # should be a list
for i, table in enumerate(tables, start=1):
    lst = {
        'table_name': f'table_{i}',  
        'duplicates': table.duplicated().sum(),
        'columns': table.columns.nunique() 
    }
    cleaning_features_table.append(lst)

    cols_features = {}
    for col in table.columns:
        col_dtype = table[col].dtype
        col_summary = {
            'dtype': str(col_dtype),
            'unique_entries': table[col].nunique()
        }
        if np.issubdtype(col_dtype, np.number):
            col_summary['sum_numeric_col'] = table[col].sum()

        cols_features[col] = col_summary

    # Display summary
    cols_features


pd.DataFrame(cols_features).T.reset_index()
# duplicates = new_df1.duplicated().sum()
# duplicates

,index,dtype,unique_entries,sum_numeric_col
0,new_id,int64,26,361
1,new_names,object,6,NaN
2,new_Amount,float64,30,8360.01
3,new_date,object,17,NaN


In [65]:

pd.DataFrame(cleaning_features_table)

,table_name,duplicates,columns
0,table_1,0,4
1,table_2,0,4
